In [21]:
import os
import json
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import matplotlib.pyplot as plt


In [22]:
# Load JSON walk data into a DataFrame
def load_walk_json(filepath):
    with open(filepath) as f:
        data = json.load(f)
    return pd.DataFrame(data)


In [23]:
# Extract gait features from accelerometer magnitude
def extract_features(df):
    df_accel = df[df['type'] == 'accel']
    x, y, z = df_accel['x'].values, df_accel['y'].values, df_accel['z'].values
    magnitude = np.sqrt(x**2 + y**2 + z**2)

    peaks, _ = find_peaks(magnitude, distance=5)

    return {
        'mean_mag': magnitude.mean(),
        'std_mag': magnitude.std(),
        'max_mag': magnitude.max(),
        'min_mag': magnitude.min(),
        'num_peaks': len(peaks),
        'peak_mean': magnitude[peaks].mean() if len(peaks) else 0
    }


In [26]:
import json, os
import numpy as np

walk_dir = "walks"
X = []
y = []
labels = []

for filename in os.listdir(walk_dir):
    if filename.endswith(".json"):
        with open(os.path.join(walk_dir, filename), "r") as f:
            data = json.load(f)
            accel = [d for d in data if d["type"] == "accel"]
            gyro = [d for d in data if d["type"] == "gyro"]

            # Simple features: average acceleration and rotation
            features = []
            for group in (accel, gyro):
                for axis in ("x", "y", "z"):
                    values = [d[axis] for d in group]
                    features.append(np.mean(values) if values else 0)
                    features.append(np.std(values) if values else 0)

            X.append(features)
            label = 1 if "user" in filename.lower() else 0
            y.append(label)
            labels.append(label)

from collections import Counter
print("✅ Loaded files:", os.listdir(walk_dir))
print("✅ Label counts:", Counter(y))


✅ Loaded files: ['walks_other.json', 'walk_user.json', 'walk_user2.json']
✅ Label counts: Counter({1: 2, 0: 1})


In [28]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import numpy as np

# Final check
unique_labels = np.unique(y)
if len(unique_labels) < 2:
    raise ValueError(f"Need at least 2 classes to train SVM. Found only: {unique_labels}")

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Train classifier
clf = SVC(kernel='rbf', probability=True)
clf.fit(X_train, y_train)

# Evaluate
accuracy = clf.score(X_test, y_test)
print("✅ Model Accuracy:", accuracy)


✅ Model Accuracy: 1.0


In [29]:
import joblib
joblib.dump(clf, "gait_model.pkl")
print("✅ Model saved as gait_model.pkl")


✅ Model saved as gait_model.pkl
